# DeepLabCut style dataset parser

### The generated output includes the following annotation data:
* Classes
* 2D keypoint data _(poses)_

### Example application(s) (as demonstrated in Plum et al. 2023):
* [DeepLabCut](https://www.mackenziemathislab.org/deeplabcut) _(single and multi-animal pose estimation)_

### Output structure:
* target_dir
    * CollectedData_SCORER.csv _(pose annoation file)_
    * CollectedData_SCORER.h5 _(pose annoation file)_
    * all generated images
    
### Notes:

* For ease of use, consider pointing the _**target_dir**_ to the _**"labelled_data"**_ directory in your created **DeepLabCut** project

* **WARNING :** Restrict the **Colony size** to a **maxmimum of 20 individuals**, otherwise the generation of datasets will fail, as the number of necessary columns would exceed the 64kb Metadata limit of the exported HDF5 file!

* Ensure, that your **GPU** has sufficient **memory** for the chosen resolution! E.g. at 1024 px X 1024 px, training on an RTX 2080 Ti, set the **batchsize** to 4 in the **pose_cfg.yaml** file of your model.

In [ ]:
import cv2
import pathlib
import json

import numpy as np
import pandas as pd
import matplotlib as plt

from os import listdir
from os.path import isfile, join

### Required parameters

Specify the location of your **generated dataset** and in which **output directory** you wish to save it.

**Notes:**
* do not include trailing forward slashes in your paths (see examples below)
* Your **dataset** name should **NOT include underscores** as they are used to separate passes into their categories. Instead, use hyphens in your naming convention where applicable.

In [ ]:
# define location of dataset and return all files

# single-animal example
dataset_location = "../example_data/input-single"
target_dir = "../example_data/DLC_SINGLE"

# multi-animal example
#dataset_location = "../example_data/input-multi"
#target_dir = "../example_data/DLC_MULTI"

SCORER = "Fabi"

# specify which labels to ignore. By default, all keypoints are written into the dataset
# in this example we omit all keypoints relating to wings. Refer to the base_rig documentation for naming conventions
omit_labels = ['w_1_l', 'w_1_l_end', 'w_2_l', 'w_2_l_end', 'w_1_r', 'w_1_r_end', 'w_2_r', 'w_2_r_end', 'root']

### Optional parameters

In [ ]:
# set True to show processing results for each image (disables parallel processing)
DEBUG = False

# we can optionally remove occluded points from the dataframe
EXCLUDE_OCCLUDED_KEYPOINTS = True

enforce_single_class = False # overwrites multiple classes and groups all instances as one

The following lines will load the generated dataset from your drive and prepare it for the multi-threaded parsing process

In [ ]:
all_files = [f for f in listdir(dataset_location) if isfile(join(dataset_location, f))]
all_files.sort()

# next, sort files into images, depth maps, segmentation maps, data, and colony info
# we only need the location and name of the data files, as all passes follow the same naming convention
dataset_data = []
dataset_img = []
dataset_ID = []
dataset_depth = []
dataset_norm = []
dataset_colony = None

for file in all_files:
    loc = dataset_location + "/" + file
    file_info = file.split("_")
    
    if file_info[1] == "BatchData":
        dataset_colony = loc
        
    elif len(file_info) == 2:
        # images are available in various formats, but annotation data is always written as json files
        if file_info[-1].split(".")[-1] == "json":
            dataset_data.append(loc)
        else:
            dataset_img.append(loc)
            
    elif file_info[2].split(".")[0] == "ID":
        dataset_ID.append(loc)
    elif file_info[2].split(".")[0]  == "depth":
        dataset_depth.append(loc)
    elif file_info[2].split(".")[0]  == "norm":
        dataset_norm.append(loc)
        
print("Found",len(dataset_data),"samples...")

# next sort the colony info into its IDs to determine the colony size and individual scales
# Opening colony (BatchData) JSON file
colony_file = open(dataset_colony)
 
# returns JSON object as a dictionary
colony = json.load(colony_file)
colony_file.close()


""" !!! requires IDs, model names, scales !!! """


if not enforce_single_class:
    # get provided classes to create a dictionary of class IDs and class names
    subject_class_names = np.unique(np.array(colony["Subject Variations"]))
    subject_classes = {}
    for id,sbj in enumerate(subject_class_names):
        subject_classes[str(sbj)] = id
else:
    subject_class_names = np.array([0])
    subject_classes = {"insect" : 0}

print("\nA total of",len(subject_class_names),"unique classes have been found.")
print("The classes and respective class IDs are:\n",subject_classes,"\n")


print("Loaded colony file with seed", colony['Seed']) #,"and",len(colony['ID']),"individuals.")
    
if len(colony['Subject Variations']) > 1:
    multi_animal = True
    print("Generating MULTI-animal dataset! Containing",len(colony['Subject Variations']),"individuals")
else:
    multi_animal = False
    print("Generating SINGLE-animal dataset!")

As there may be animals for which we don't use all bones we can return a list of all labels and exclude the respective locations from the pose data. As all animals use the same convention, we can simply read in one example and remove the corresponding indices from all animals.

In [ ]:
# loading the first entry of first iteration file to retrieve skeleton info
exp_file = open(dataset_data[0])
exp_data = json.load(exp_file)
exp_file.close()

# for simplicity we'll assume that at this stage all subjects use the same armature and therefore report the same keypoints
first_entry_key = list(exp_data["iterationData"]["subject Data"][0].keys())[0]
labels = list(exp_data["iterationData"]["subject Data"][0][first_entry_key]["keypoints"].keys())

# show all used labels:
print("\nAll labels:",labels)

print("\nOmitting labels:", omit_labels)

# removing all occurences of omitted labels from the labels list to be used as keys below
labels = [x for x in labels if x not in omit_labels]

print("\nFinal labels:",labels)

Now let us create a big array to store all our dataset info and then later convert it and save it all to the desired **.csv** and **.h5** files for DeepLabCut to read.

In [ ]:
all_points = np.zeros((len(dataset_data), (len(colony['Subject Variations'])*(len(labels)*2))))
#	- scorer   #(just one, the only scorer is the generator)
#	- - individuals
#	- - - bodyparts
#	- - - - coords

print("Number of loaded samples:",len(dataset_data))
print("Colony size:",len(colony['Subject Variations']))
print("body parts:",int(len(labels)),"\n")
print("Resulting in an array of shape:",all_points.shape)

output_file_names = ["" for i in range(len(dataset_data))]

With all dataset related parameters configured, we have provided a multi-threaded parsing solution below to minimise the processing time it takes to bring the entire dataset into the required output format. Currently, we instanciate one processing thread per (virtual) CPU core but you can adjust this value if you wish by changing:

```
threadList_export = createThreadList(#NumDesiredThreads)
```

**Note:** To see the process of mask generation from ID passes in action, set the **DEBUG** mode to **"True"**. This will however slow down the processing speed considerably and only run in single-threaded mode!

In [ ]:
# create unique colours for each ID
import numpy as np
import time

# alright. Let's take it from the top and fucking multi-thread this.
import threading
import queue
import sys
import os

def fix_bounding_boxes(coords,max_val = [1024,1024]):
    # fix bounding box coordinates so they do not reach beyond the image
    fixed_coords = []
    for c, coord in enumerate(coords):
        if c == 0 or c == 2:
            max_val_temp = max_val[0]
        else:
            max_val_temp = max_val[1]
            
        if coord >= max_val_temp:
            coord = max_val_temp
        elif coord <= 0:
            coord = 0
        
        fixed_coords.append(int(coord))
        
    return fixed_coords

def getThreads():
    """ Returns the number of available threads on a posix/win based system """
    if sys.platform == 'win32':
        return int(os.environ['NUMBER_OF_PROCESSORS'])
    else:
        return int(os.popen('grep -c cores /proc/cpuinfo').read())

class exportThread(threading.Thread):
    def __init__(self, threadID, name, q):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.q = q

    def run(self):
        print("Starting " + self.name)
        process_detections(self.name, self.q)
        print("Exiting " + self.name)
        
def createThreadList(num_threads):
    threadNames = []
    for t in range(num_threads):
        threadNames.append("Thread_" + str(t))

    return threadNames

def process_detections(threadName, q):
    while not exitFlag:
        queueLock.acquire()
        if not workQueue.empty():
            
            data_input = q.get()
            i, data_loc, img, ID = data_input
            queueLock.release()
            
            display_img = cv2.imread(img)
            display_img_orig = display_img.copy()
            
            # compute visibility for each individual
            seg_img = cv2.imread(ID)
            seg_img_display = seg_img.copy()
            
            data_file = open(data_loc)
            # returns JSON object as a dictionary
            data = json.load(data_file)
            data_file.close()
            
            img_shape = display_img.shape
            
            # only add images that contain visibile individuals
            is_empty = True
            
            img_name = target_dir + "/" + img.split('/')[-1][:-4] + "_synth" + ".png"
            # write the file path to the all_points array
            output_file_names[i] = "labeled-data/" + str(os.path.basename(target_dir)) + "/" + str(os.path.basename(img))[:-4] + "_synth" + ".png"

            img_info = []
                
            # check if the size of the image and segmentation pass match
            if img_shape != seg_img.shape:
                print("Size mismatch of image and segmentation pass for sample",data_input[1].split("/")[-1],"!")
                incorrectly_formatted_images.append(i)
            else:
                for individual in data["iterationData"]["subject Data"]:
                    ind_key = list(individual.keys())[0]
                    ind_ID = int(ind_key)
                    # WARNING ID numbering begins at 1

                    fontColor = (int(ID_colours[ind_ID,0]),
                                 int(ID_colours[ind_ID,1]),
                                 int(ID_colours[ind_ID,2]))
                    
                    bbox_orig = [individual[ind_key]["2DBounds"]["xmin"],
                                 individual[ind_key]["2DBounds"]["ymin"],
                                 individual[ind_key]["2DBounds"]["xmax"],
                                 individual[ind_key]["2DBounds"]["ymax"]]
                    
                    bbox = fix_bounding_boxes(bbox_orig, max_val=display_img.shape)
                    
                    # only process an individual if its bounding box width and height are not zero
                    if bbox[2] - bbox[0] == 0 or bbox[3] - bbox[1] == 0:
                        continue

                    try:
                        ID_mask = cv2.inRange(seg_img[bbox[1]:bbox[3],bbox[0]:bbox[2]], np.array([0, 0, ind_ID - 2]), np.array([0, 0, ind_ID + 2]))
                        indivual_occupancy = cv2.countNonZero(ID_mask)
                    except:
                        if len(threadList) == 1: 
                            print("Individual fully occluded:",ind_ID,"in",dataset_seg[i])
                        indivual_occupancy = 1

                    #indivual_occupancy = np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 - 1)]).all(axis = 2)) + np.count_nonzero((seg_img == [0, 0, int((individual[0]/len(colony['ID']))*255 + 1)]).all(axis = 2))
                    bbox_area = abs((bbox[2] - bbox[0]) * (bbox[3] - bbox[1])) + 1
                    bbox_occupancy = indivual_occupancy / bbox_area
                    #print("Individual", individual[0], "with bounding box occupancy ",bbox_occupancy)

                    #cv2.putText(display_img, "ID: " + str(int(individual[0])), (bbox[0] + 10,bbox[3] - 10), font, fontScale, fontColor, lineType)
                    if bbox_occupancy > visibility_threshold:
                        # let's binarise the image and dilate it to make sure all points that visible are found
                        if not multi_animal:
                            seg_bin = cv2.inRange(seg_img, np.array([0, 0, 1]), np.array([0,0, 3]))
                        else:
                            seg_bin = cv2.inRange(seg_img, np.array([0, 0, ind_ID - 1]), np.array([0, 0, ind_ID + 1]))
                        kernel = np.ones((5,5), np.uint8)
                        seg_bin_dilated = cv2.dilate(seg_bin,kernel,iterations = 2)
                        if DEBUG:
                            cv2.imshow("dilated mask",seg_bin_dilated)
                            cv2.waitKey(0)

                        for point in range(len(labels)):
                            # get rid of all invalid points first. Those should simply stay NaN in the array
                            if individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] > img_shape[0] or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] < 0 or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] > img_shape[1] or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] < 0:
                                continue
                            else:
                                # now throw the coordinates to the correct location
                                out_row = i
                                out_column = ((ind_ID - 1) * (len(labels) ) + point) * 2
                                # exclude negative keypoints
                                if individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] < 0.1 or individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] < 0.1:
                                    individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] = 0 # X
                                    individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] = 0 # Y
                                # exlucde occluded keypoints by checking their visibility in the segmentation map   
                                if EXCLUDE_OCCLUDED_KEYPOINTS:
                                    x_temp = int(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"])
                                    y_temp = int(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"])           
                                    if seg_bin_dilated[y_temp,x_temp] == 0:  
                                        
                                        if DEBUG:
                                            display_img = cv2.circle(display_img, (x_temp,y_temp), radius=0, color=(0, 0, 255), thickness=2)
                                            cv2.imshow("missing points",display_img)
                                            cv2.waitKey(0)
                                        
                                        individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"] = 0 # X
                                        individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] = 0 # Y
                                all_points[out_row][out_column] = round(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["x"], 1) # X
                                all_points[out_row][out_column + 1] = round(individual[ind_key]["keypoints"][labels[point]]["2DPos"]["y"] ,1) # Y

                cv2.imwrite(img_name, display_img)
            
        else:
            queueLock.release()
            
# setup as many threads as there are (virtual) CPU cores
exitFlag = 0
# only use a fourth of the number of CPUs for export as hugin and enfuse utilise multi core processing in part
if DEBUG:
    threadList = createThreadList(1)
else:
    threadList = createThreadList(getThreads())
print("Using", len(threadList), "threads for export...")
queueLock = threading.Lock()

# define paths to all images and set the maximum number of items in the queue equivalent to the number of images
workQueue = queue.Queue(len(dataset_img))
threads = []
threadID = 1

# keep track of all incorrectly formatted images to remove them after iterating over all entries
incorrectly_formatted_images = []

np.random.seed(seed=1)
ID_colours = np.random.randint(255, size=(255, 3))

font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 0.5
lineType = 2

# we can additionally plot the points in the data files to check joint locations
plot_joints = True

# remember to define an export folder when saving out your dataset
generate_dataset = True

# determine the proportion of a bounding box that needs to be filled before considering the visibility as too low
# WARNING: At the moment the ID shown in segmentation maps does not always correspond to the ID in the data file (off by 1)
visibility_threshold = 0.05

timer = time.time()

# Create new threads
for tName in threadList:
    thread = exportThread(threadID, tName, workQueue)
    thread.start()
    threads.append(thread)
    threadID += 1

# Fill the queue with samples
queueLock.acquire()
for i, (data, img, ID) in enumerate(zip(dataset_data , dataset_img, dataset_ID)):
    workQueue.put([i, data, img, ID])
queueLock.release()

# Wait for queue to empty
while not workQueue.empty():
    pass

# Notify threads it's time to exit
exitFlag = 1

# Wait for all threads to complete
for t in threads:
    t.join()
print("Exiting Main export Thread")

# close all windows if they were opened
cv2.destroyAllWindows()

# now, remove all incorrectly formatted images from the points and file list
all_points = np.delete(all_points, incorrectly_formatted_images ,axis=0)
for r, rem_img in enumerate(incorrectly_formatted_images):
    del output_file_names[rem_img - r]

print("Total time elapsed:",time.time()-timer,"seconds")

Now, dump it all into one **DLC-conform pandas (.h5)** file

In [ ]:
# next create the required hierarchy
scorer = [SCORER for i in range(len(all_points[0]))]
individuals = []

for ind in range(len(colony['Subject Variations'])):
    ### UPDATE ONCE ALL COLONY INFO IS INCLUDED ###
    individual = ["id_" + str(ind) +"_num_" + str(ind) for i in range(int((len(labels))*2))]
    individuals.extend(individual)


bodyparts_filtered = [i for j in labels for i in [j]*2]
bodyparts = []

for i in range(len(colony['Subject Variations'])):
    bodyparts.extend(bodyparts_filtered)

coords = []
for i in range(int(len(all_points[0])/2)):
    coords.extend(["x","y"])

Now that all elemts for the **Multi-Index** hierachy are defined, we can combine them into the **final dataframe**  

In [ ]:
if multi_animal:
    categories = [scorer, individuals, bodyparts, coords]
    categories_tuples = list(zip(*categories))
    columns = pd.MultiIndex.from_tuples(categories_tuples, names=["scorer",
                                                               "individuals",
                                                               "bodyparts",
                                                               "coords"])
else:
    categories = [scorer, bodyparts, coords]
    categories_tuples = list(zip(*categories))
    columns = pd.MultiIndex.from_tuples(categories_tuples, names=["scorer",
                                                               "bodyparts",
                                                               "coords"])
    
final_dataframe = pd.DataFrame(all_points, index = output_file_names, columns=columns)
# convert all zeros to NaN
final_dataframe = final_dataframe.replace(0, np.nan)

Display the output dataframe...

In [ ]:
final_dataframe

...and write it all out.

In [ ]:
final_dataframe.to_csv(os.path.join(target_dir, "CollectedData_" + SCORER + ".csv"))

# IF the function below fails, this is likely due to exceeding the number of columns supported by HDF5 files!
# Restrict the number of simulated animals to < 20 if the goal is to train a DLC network

final_dataframe.to_hdf(
    os.path.join(target_dir, "CollectedData_" + SCORER + ".h5"),
    "df_with_missing",
    format="table",
    mode="w")
    